Running this code will compare 21 heat index algorithms from Anderson et al (2013) to Lu and Romps (2022)
If using hourly-level data, expect considerable waiting times (8+ hours)

# Basic loading

In [1]:
%run Step_5_Data_Analysis_backend.ipynb
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import iris
import iris.coord_categorisation
import iris.analysis
import matplotlib.pyplot as plt

In [ ]:
# tas: temperature at surface (in ˚C)
# rh: relative humidity (in %)
# td: dew point temperature (in ˚C)
# es: water vapor pressure (in kilopascals)

# Heat Index 22 is the Lu and Romps heat index cube

location_name = 'Limpopo'
start_year = 1950
end_year = 2024

# Specify an output folder for figures
output_folder = f'{os.getcwd()}/{location_name}_outputs'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

unprocessed_output_folder = f'{output_folder}/basic_plots_for_unprocessed_data'
if not os.path.exists(unprocessed_output_folder):
    os.makedirs(unprocessed_output_folder)

# Specify shapefile to work with
shapefile = gpd.read_file('/Users/maxwhite/Documents/Met_Office_Work/Heat_index_Algorithms/Data/Limpopo/Limpopo_Boundaries.geojson')

heat_indeces_cubelist, tas_cube, rh_cube, td_cube, es_cube = load_my_files()

# Add day of year coordinate to cubes if it doesn't already exist
for cube in heat_indeces_cubelist:
    try:
        cube.coord('day_of_year')
    except iris.exceptions.CoordinateNotFoundError:
        iris.coord_categorisation.add_day_of_year(cube, 'time', name='day_of_year')

In [ ]:
add_dimenions_to_my_cubes()

In [ ]:
# CREATING A TEMPORARY SUBSET
heat_indeces_cubelist_subset = iris.cube.CubeList([])
for cube in heat_indeces_cubelist:
    cube_subset = cube.extract(iris.Constraint(time=lambda cell: 2020 <= cell.point.year <= 2022))
    heat_indeces_cubelist_subset.append(cube_subset)

heat_indeces_cubelist = heat_indeces_cubelist_subset

In [ ]:
create_yearly_aggregates() # Creates season year aggregates of the tas, rh, td, and es cubes
create_overall_aggregates() # Creates mappable aggregates of the tas, rh, td, and es cubes

In [ ]:
collapse_cubes_for_time_series() # Spatially collapses the cubes for time series analysis
collapse_cubes_for_maps() # Temporally collapses the cubes for map analysis


In [ ]:
collapse_air_temperature_for_time_series() # Collapses the air temperature cube for time series analysis to allow comparison
collapse_air_temperature_for_maps() # Collapses the air temperature cube for map analysis to allow comparison

# Basic descriptive plots

In [ ]:
plot_my_region() # Plots the region specified by the shapefile

In [ ]:
plot_yearly_average_tas_and_rh() # Plots the yearly mean, min, and max values of tas, rh, td, and es

In [ ]:
plot_monthly_average_tas_and_rh() 

In [ ]:
plot_hourly_average_tas_and_rh()

In [ ]:
plot_min_mean_max_analysis_maps() # Plots the mean analysis maps for tas, rh, td, and es

In [ ]:
write_summary_statistics() # Writes summary statistics to a csv file (mean, min, max, std, variance)

# 2. Comparing the algorithms

## Threshold analyses

In [ ]:
# Creating masked versions of the cube, containing only data ≥ 298 for computational efficiency of below tasks
masked_hourly_data_cubelist = heat_indeces_cubelist.copy()
for cube in masked_hourly_data_cubelist:
    cube.data = np.ma.masked_where(cube.data < 298, cube.data)

In [ ]:
# Define the thresholds
thresholds = [298, 345, 357, 366]

# Create an empty DataFrame to store the results
hourly_counts_per_threshold_df = pd.DataFrame(columns=['Algorithm'] + [f'Count_{threshold}' for threshold in thresholds])

hourly_counts_per_threshold_df['Algorithm'] = [cube.long_name for cube in masked_hourly_data_cubelist]
hourly_counts_per_threshold_df

for i, cube in enumerate(masked_hourly_data_cubelist):
    count_above_298 = (cube.data >= 298).sum()
    hourly_counts_per_threshold_df.loc[i, 'Count_298'] = count_above_298

for i, cube in enumerate(masked_hourly_data_cubelist):
    count_above_345 = (cube.data >= 345).sum()
    hourly_counts_per_threshold_df.loc[i, 'Count_345'] = count_above_345

for i, cube in enumerate(masked_hourly_data_cubelist):
    count_above_357 = (cube.data >= 357).sum()
    hourly_counts_per_threshold_df.loc[i, 'Count_357'] = count_above_357

for i, cube in enumerate(masked_hourly_data_cubelist):
    count_above_366 = (cube.data >= 366).sum()
    hourly_counts_per_threshold_df.loc[i, 'Count_366'] = count_above_366

print(hourly_counts_per_threshold_df)
hourly_counts_per_threshold_df.to_csv(f'{output_folder}/hourly_counts_per_threshold.csv', index=False)

In [ ]:
# Plotting the bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Set the positions and width for the bars (REQUIRED FOR PLOTTING MULTIPLE BARS FOR THE SAME VARIABLE)  
positions = range(len(hourly_counts_per_threshold_df))

# Plot each threshold as a separate set of bars
for i, threshold in enumerate(thresholds):
    ax.bar([position + bar_width * i for position in positions], # Shifting the bars to allow plotting per threshold
           hourly_counts_per_threshold_df[f'Count_{threshold}'], 
           width=0.2,
           label=f'Count > {threshold}')

# Set the x-ticks and labels
ax.set_xticks([position + 0.2 * (len(thresholds) / 2 - 0.5) for position in positions]) ##TODO: improve this positioning
ax.set_xticklabels(hourly_counts_per_threshold_df['Algorithm'])

# Add labels and title
ax.set_xlabel('Algorithm')
ax.set_ylabel('Count of Hours')
ax.set_title(f'Count of Hours Above Thresholds {start_year}–{end_year}')
ax.legend()

# Show the plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### Daily mean level

In [ ]:
daily_mean_heat_indeces_cubelist = iris.cube.CubeList([])
for cube in heat_indeces_cubelist:
    daily_mean = cube.aggregated_by(['year', 'day_of_year'], iris.analysis.MEAN)
    daily_mean_heat_indeces_cubelist.append(daily_mean)

# Create an empty DataFrame to store the results
daily_counts_per_threshold_df = pd.DataFrame(columns=['Algorithm'] + [f'Count_{threshold}' for threshold in thresholds])

daily_counts_per_threshold_df['Algorithm'] = [cube.long_name for cube in heat_indeces_cubelist]
daily_counts_per_threshold_df

for i, cube in enumerate(daily_mean_heat_indeces_cubelist):
    count_above_298 = (cube.data >= 298).sum()
    daily_counts_per_threshold_df.loc[i, 'Count_298'] = count_above_298

for i, cube in enumerate(daily_mean_heat_indeces_cubelist):
    count_above_345 = (cube.data >= 345).sum()
    daily_counts_per_threshold_df.loc[i, 'Count_345'] = count_above_345

for i, cube in enumerate(daily_mean_heat_indeces_cubelist):
    count_above_357 = (cube.data >= 357).sum()
    daily_counts_per_threshold_df.loc[i, 'Count_357'] = count_above_357

for i, cube in enumerate(daily_mean_heat_indeces_cubelist):
    count_above_366 = (cube.data >= 366).sum()
    daily_counts_per_threshold_df.loc[i, 'Count_366'] = count_above_366

daily_counts_per_threshold_df

In [ ]:
## TODO: Days above threshold for each algorithm

In [ ]:
# # Define the thresholds
# thresholds = [298, 345, 357, 366]

# # Create an empty DataFrame to store the results
# daily_counts_per_threshold_df = pd.DataFrame(columns=['Algorithm'] + [f'Count_{threshold}' for threshold in thresholds])

# daily_counts_per_threshold_df['Algorithm'] = [cube.long_name for cube in heat_indeces_cubelist]


# for i, cube in enumerate(heat_indeces_cubelist):
#     count_above_298 = (cube.data > 298).sum()
#     daily_counts_per_threshold_df.loc[i, 'Count_298'] = count_above_298

# for i, cube in enumerate(heat_indeces_cubelist):
#     count_above_345 = (cube.data > 345).sum()
#     daily_counts_per_threshold_df.loc[i, 'Count_345'] = count_above_345

# for i, cube in enumerate(heat_indeces_cubelist):
#     count_above_357 = (cube.data > 357).sum()
#     daily_counts_per_threshold_df.loc[i, 'Count_357'] = count_above_357

# for i, cube in enumerate(heat_indeces_cubelist):
#     count_above_366 = (cube.data > 366).sum()
#     daily_counts_per_threshold_df.loc[i, 'Count_366'] = count_above_366

# daily_counts_per_threshold_df

## Basic comparisons between heat index algorithms

In [ ]:
plot_yearly_mean_values_from_heat_indexes() # Plots the yearly mean values of the heat indexes

In [ ]:
create_table_of_differences_between_heat_index_and_air_temperature() # Creates a table of differences between heat index and air temperature

In [ ]:
plot_difference_from_air_temperature() # Plots the difference from air temperature

In [ ]:
plot_mean_differences_from_Lu_and_Romps_for_each_index() # Plots the mean differences from Lu and Romps for each index
print("REMEMBER! Lu and Romps is basically 0 anyway so the plots will look similar")

In [ ]:
create_table_of_differences_between_heat_index_and_Lu_and_Romps() # Creates a table of differences between heat index and Lu and Romps

In [ ]:
plot_heat_index_comparison_maps() # Plots the heat index comparison maps

In [ ]:
plot_heat_index_maps_relative_to_Lu_and_Romps() # Plots the heat index maps relative to Lu and Romps

In [ ]:
normalise_cubelist() # Normalises the cube list

In [ ]:
plot_normalized_algorithms()